### Instance generator

In [1]:
using Random

function generate_random_symmetric_tsp(n::Int, max_cost::Int=100)
    C = zeros(Int, n, n)
    for i in 1:n-1
        for j in i+1:n
            C[i, j] = rand(1:max_cost)
            C[j, i] = C[i, j]
        end
    end
    return C
end

# Generate 5 random TSP instances with n=10
tsp_instances = [generate_random_symmetric_tsp(10) for _ in 1:5]

# Print each instance
for (idx, mat) in enumerate(tsp_instances)
    println("Instance $idx:")
    println(mat)
    println()
end

Instance 1:
[0 48 45 89 98 58 2 83 13 82; 48 0 48 22 69 32 49 48 94 11; 45 48 0 78 58 81 99 97 64 37; 89 22 78 0 43 34 27 86 31 60; 98 69 58 43 0 71 39 89 5 20; 58 32 81 34 71 0 78 20 96 91; 2 49 99 27 39 78 0 18 63 75; 83 48 97 86 89 20 18 0 45 4; 13 94 64 31 5 96 63 45 0 42; 82 11 37 60 20 91 75 4 42 0]

Instance 2:
[0 69 80 75 31 84 24 35 86 65; 69 0 98 88 87 4 76 17 69 65; 80 98 0 36 2 56 48 87 25 78; 75 88 36 0 42 4 98 81 73 35; 31 87 2 42 0 25 68 45 55 24; 84 4 56 4 25 0 87 65 42 46; 24 76 48 98 68 87 0 78 82 60; 35 17 87 81 45 65 78 0 74 61; 86 69 25 73 55 42 82 74 0 3; 65 65 78 35 24 46 60 61 3 0]

Instance 3:
[0 38 63 93 89 86 69 23 85 35; 38 0 24 69 60 10 55 75 2 12; 63 24 0 30 56 22 23 44 80 68; 93 69 30 0 5 74 41 77 28 38; 89 60 56 5 0 53 39 12 100 66; 86 10 22 74 53 0 32 96 28 49; 69 55 23 41 39 32 0 62 65 26; 23 75 44 77 12 96 62 0 96 90; 85 2 80 28 100 28 65 96 0 44; 35 12 68 38 66 49 26 90 44 0]

Instance 4:
[0 4 64 93 7 66 33 38 96 74; 4 0 5 74 11 21 24 39 31 67; 64 5 

### (a) MTZ formulation

In [2]:
using JuMP, Gurobi, Statistics

In [4]:
function solve_mtz_tsp(C::Matrix{Int})
    n = size(C, 1)
    model = Model(Gurobi.Optimizer)
    
    set_silent(model)

    # Binary decision variables x[i,j]
    @variable(model, x[1:n, 1:n], Bin)
    # MTZ auxiliary variables with bounds 2 <= u[i] <= n for i=2..n
    @variable(model, 2 <= u[2:n] <= n)

    # Objective: minimize total cost
    @objective(model, Min, sum(C[i,j] * x[i,j] for i in 1:n, j in 1:n))

    # Degree constraints: one in, one out per city
    @constraint(model, [i=1:n], sum(x[i, j] for j in 1:n) == 1)
    @constraint(model, [j=1:n], sum(x[i, j] for i in 1:n) == 1)

    # MTZ subtour elimination
    @constraint(model, [i=2:n, j=2:n; i != j], u[i] - u[j] + n * x[i, j] <= n - 1)

    # No self-loops
    @constraint(model, [i=1:n], x[i, i] == 0)

    # Solve and return time & tour length
    elapsed = @elapsed optimize!(model)
    tour_length = objective_value(model)
    return elapsed, tour_length
end

solve_mtz_tsp (generic function with 1 method)

### (b) DFJ formulation + lazy constraints

#### Helper Methods

In [ ]:
# Helper to find subtours in a solution matrix
function find_subtours(sol::Matrix{Int})

    n = size(sol,1)
    visited = falses(n)
    subtours = Vector{Vector{Int}}()

    for i in 1:n
        if !visited[i]

            queue = [i]
            comp = Int[]
            visited[i] = true

            while !isempty(queue)
                u = popfirst!(queue)
                push!(comp,u)

                for v in 1:n
                    if sol[u,v] == 1 && !visited[v]
                        visited[v] = true
                        push!(queue,v)
                    end
                end

            end

            push!(subtours, comp)

        end
    end
    
    return subtours

end

#### Formulation